In [ ]:
# CREANDO LAS FUNCIONES PARA LA API

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Carga de datasets

# Se cargan los dataframes para llevar a cabo las consultas solicitadas en el readme del proyecto

#dfreviews_unido: contiene la información de los reviews de usuario para cada juego en formato de análisis de sentimientos. Además, están las recomendaciones o no del juego por parte de usuario, la fecha del review y sus datos (id, url)

#df_gastos_items: cantidad de items que consumidos por usuario y el precio de cada uno de los juegos que consume.

# dfdesarrolladores_recomendados: contiene un desarrollador y calcula cuantos analisis de sentimiento positivos y negativos tiene

#df_playtime_forever: la identificacion del usuario, su url, y la cantidad de horas jugadas por genero.

#df_items_developer: contiene la información relacionada con cada item de juegos como id, precio, desarrollador y año de lanzamiento.

In [3]:
df_items_developer = pd.read_csv('C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\df_items_developer_unido.csv')

df_reviews = pd.read_csv('C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\dfreviews_unido.csv')

df_gastos_items = pd.read_csv('C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\df_gastos_items_unido.csv')

dfdesarrolladores_recomendados = pd.read_csv('C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\dfdesarrolladores_recomendados_unido.csv')

df_playtime_forever = pd.read_csv('C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\df_playtime_forever_unido.csv')

In [4]:
# FUNCION DEVELOPER

#Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 

def developer(desarrollador):
    empresa_desarrolladora = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Cantidad de items por año
    items_por_año = empresa_desarrolladora.groupby('anio_lanzamiento')['item_id'].count()
    # Cantidad de elementos gratis por año
    contenido_free = empresa_desarrolladora[empresa_desarrolladora['price']==0.00].groupby('anio_lanzamiento')['item_id'].count()
    # Porcentaje items gratis por año
    porcentaje_gratis = ((contenido_free/items_por_año)*100).fillna(0).astype(int)

    result_dict = {'Cantidad anual': items_por_año.to_dict(),
                   'Porcentaje de juegos que son gratis': porcentaje_gratis.to_dict()}
    
    tabla = pd.DataFrame(result_dict)
    
    return tabla

In [7]:
developer('Ubisoft')

,Cantidad anual,Porcentaje de juegos que son gratis
2003,2,0
2004,1,0
2005,1,0
2007,2,0
2010,1,0
2014,7,0
2015,1,0
2016,20,10
2017,11,0


In [8]:
# FUNCION USERDATA

# Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews_recommend y cantidad de items consumidos.

def userdata(user_id):
    # Filtramos por usuario
    usuario = df_reviews[df_reviews['user_id'] == user_id]
    # Calculamos cantidad de dinero gastado por ese usuario
    cantidad_dinero = df_gastos_items[df_gastos_items['user_id'] == user_id]['price'].iloc[0]
    # Buscamos la cantidad de items para el usuario
    count_items = df_gastos_items[df_gastos_items['user_id'] == user_id]['items_count'].iloc[0]

    # Total de recomendaciones del usuario
    total_recomendaciones = usuario['reviews_recommend'].sum()
    # Total de reviews por todos los usuarios
    total_reviews = len(df_reviews['user_id'].unique())
    # porcentaje de recomendaciones 
    porcentaje_recomendaciones = (total_recomendaciones/total_reviews)*100

    #algunos valores pueden ser nulo, por lo que la api no los reconoce y no corre las instrucciones, por lo que agregamos un condicional para count_items
    count_items = count_items if not pd.isnull(count_items) else 0

    return {'usuario': user_id,
            'cantidad_dinero': cantidad_dinero,
            'porcentaje_recomendacion': round(porcentaje_recomendaciones,2),
            'cantidad_de_items': count_items.astype(int)}

In [9]:
userdata('--ace--')

{'usuario': '--ace--',
 'cantidad_dinero': 166.82,
 'porcentaje_recomendacion': 0.0,
 'cantidad_de_items': 44}

In [10]:
#FUNCION USERFORGENRE

# Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

def UserForGenre(genre):
    # Filtra el dataframe por el género de interés
    data_por_genero = df_playtime_forever[df_playtime_forever['genres'] == genre]
    # Agrupa el dataframe y suma la cantidad de horas jugadas
    top_user = data_por_genero.groupby('user_id')['playtime_horas'].sum().reset_index().iloc[0].to_dict()
    
    return top_user

In [12]:
UserForGenre('Indie')

{'user_id': '--000--', 'playtime_horas': 203.63333333333333}

In [13]:
# FUNCION BESTDEVELOPERYEAR

# Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado (reviews.recommend = True y comentarios positivos)

def best_developer_year(year): 
    # filtramos por año
    año_seleccionado = dfdesarrolladores_recomendados[dfdesarrolladores_recomendados['anio_lanzamiento'] == year]
    # seleccionamos solo recomendaciones y sentimiento positivo
    recomendados_positivos = año_seleccionado[(año_seleccionado['reviews_recommend'] == True) & (año_seleccionado['sentiment_analysis'] == 2)]
    # agrupamos por desarrollador y calculamos la cantidad de sentimientos positivos y recomendaciones
    agrupado = recomendados_positivos.groupby('developer')[['reviews_recommend', 'sentiment_analysis']].count()
    # creamos una nueva columna para calcular la sumatoria total de estas variables
    agrupado['suma'] = agrupado['reviews_recommend'] + agrupado['sentiment_analysis']

    top3 = agrupado['suma'].sort_values(ascending=False).nlargest(3).to_dict()

    return top3

In [14]:
best_developer_year('2009')

{'Valve': 454, 'Tripwire Interactive': 196, 'Infinity Ward,Aspyr (Mac)': 70}

In [15]:
# FUNCION DEVELOPER_REVIEWS_ANALYSIS 

# Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

def developer_reviews_analysis(desarrollador): 
    # filtro por desarrollador
    filtro_desarrollador = dfdesarrolladores_recomendados[dfdesarrolladores_recomendados['developer'] == desarrollador]
    
    sentimientos_positivos = filtro_desarrollador[filtro_desarrollador['sentiment_analysis'] == 2]['sentiment_analysis'].count()
    
    sentimientos_negativos = filtro_desarrollador[filtro_desarrollador['sentiment_analysis'] == 0]['sentiment_analysis'].count()

    diccionario = {'Desarrollador': desarrollador,
                   'Positivos': int(sentimientos_positivos),
                   'Negativos': int(sentimientos_negativos)}

    return diccionario

In [17]:
developer_reviews_analysis('Valve')

{'Valve': [{'Positivos': 3223, 'Negativos': 711}]}